In [1]:
import cv2
import pygame
import random
import numpy as np
import pyrealsense2 as rs
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
from PIL import Image
from matplotlib import pyplot as plt

pygame 2.6.0 (SDL 2.28.4, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
verts = (
  (1, -1, -1),
  (1, 1, -1),
  (-1, 1, -1),
  (-1, -1, -1),
  (1, -1, 1),
  (1, 1, 1),
  (-1, -1, 1),
  (-1, 1, 1))

edges = (
  (0,1),(0,3),(0,4),
  (2,1),(2,3),(2,7),
  (6,3),(6,4),(6,7),
  (5,1),(5,4),(5,7))

surfaces = (
  (0,1,2,3),
  (3,2,7,6),
  (6,7,5,4),
  (4,5,1,0),
  (1,5,7,2),
  (4,0,3,6))

colors = (
  (1,0,0),
  (0,1,0),
  (0,0,1),
  (1,1,0),
  (0,1,1),
  (1,0,1))    

In [3]:
width, height = 1200, 800
dwidth, dheight = 1280, 720 # 640 480 
imageScale = 200
frame, gray, resized = 0, 0, 0
capture, pipeline, align = 0, 0, 0

displayAxis = False
displayPlane = False
displayInformation = True
displayRealSensor = True
displayHeatCamera = True
displayTracking = True
captureFlag = True
thresholdFlag = True
debugCameraFlag = False

projection = True
button = False
loop = True
clock, frameCount = 0, 0

skip, pointSize, faceType = 4, 2, 0
ym = 5.4                                    # display image y margin
rotX, rotY = 0, 0
scale, zScale = 12., 1.                     # opengl display scale, z axis scale

objSx, objSy = 0, 0                         # face object start x, y
objWidth, objHeight = 0, 0                  # face object width, height
minDepth, maxDepth, meanDepth = 0, 0, 0     # face object min, max, mean depth 

msg = "..."
flirMsg = "..."
font, fontSize = 0, 18
fRate, size = 0.1, 5
mouseX, mouseY, pmouseX, pmouseY = 0, 0, 0, 0

eye = (0., 0., 10.)
center = (0., 0., 0.)
fps, delay = 30, 1 # 33

depthImage, colorImage = 0, 0
dmaskImage, depthColormap, convertDepthImage = 0, 0, 0

In [15]:

def myInit(_dwidth, _dheight):
    global width, height, dwidth, dheight, objWidth, objHeight
    global font, fontSize, loop, frameCount, clock
    loop = True
    frameCount = 0

    pygame.init()
    clock = pygame.time.Clock()
    dwidth = _dwidth
    dheight = _dheight
    objWidth = dwidth
    objHeight = dheight

    pygame.display.set_caption('jluFace3D Analysis with realSensor')
    screen = pygame.display.set_mode((width, height), DOUBLEBUF|OPENGL) # FULLSCREEN
    font = pygame.font.SysFont("courier new", fontSize, bold=True) #'arial'
    myTexture()

def myTexture():
    image = Image.open("simage1.jpg") #realsense-logo.png
    img_data = np.array(list(image.getdata()), np.uint8)
    updateImage(img_data, image.width, image.height, 1) #dwidth, dheight
    
    glBindTexture(GL_TEXTURE_2D, 1)
    glTexParameteri(GL_TEXTURE_2D,GL_TEXTURE_MAG_FILTER, GL_NEAREST)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_NEAREST)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, 
        image.width, image.height, 0, GL_RGB, GL_UNSIGNED_BYTE, img_data)
    glTexEnvf(GL_TEXTURE_ENV, GL_TEXTURE_ENV_MODE, GL_REPLACE)

    glBindTexture(GL_TEXTURE_2D, 2)
    glTexParameteri(GL_TEXTURE_2D,GL_TEXTURE_MAG_FILTER, GL_NEAREST)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_NEAREST)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, 
        image.width, image.height, 0, GL_RGB, GL_UNSIGNED_BYTE, img_data)
    glTexEnvf(GL_TEXTURE_ENV, GL_TEXTURE_ENV_MODE, GL_REPLACE)

    glBindTexture(GL_TEXTURE_2D, 3)
    glTexParameteri(GL_TEXTURE_2D,GL_TEXTURE_MAG_FILTER, GL_NEAREST)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_NEAREST)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, 
        image.width, image.height, 0, GL_RGB, GL_UNSIGNED_BYTE, img_data)
    glTexEnvf(GL_TEXTURE_ENV, GL_TEXTURE_ENV_MODE, GL_REPLACE)


def myReshape():
    global projection, size, width, height
    glClearColor (.75, .75, .75, 0.0)
    glViewport(0, 0, width, height)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    if projection:
        gluPerspective(60., width/height, .1, 10000.)
    else:
        glOrtho(-size, size, -size, size, -size, size)
    glMatrixMode(GL_MODELVIEW)
    

def updateImage(image, width, height, index):
    glBindTexture(GL_TEXTURE_2D, index)
    # img_data = np.array(list(image)) #, np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, width, height, 0, GL_BGR, GL_UNSIGNED_BYTE, image) #img_data
    

def drawText(x, y, text, gray): 
    textSurface = font.render(text, True, (255, 255, 255, 255), (gray, gray, gray, 255))
    textData = pygame.image.tostring(textSurface, "RGBA", True)
    glWindowPos2d(x, y)
    glDrawPixels(textSurface.get_width(), textSurface.get_height(), GL_RGBA, GL_UNSIGNED_BYTE, textData)


def myAxis():
    global objWidth, objHeight
    global minDepth, maxDepth, meanDepth

    f = 1000
    z1 = minDepth - meanDepth
    z2 = maxDepth - meanDepth
    glPushMatrix()
    glTranslatef(-objWidth/f/2, objHeight/f/2, 0)
    glScalef(1/f, -1/f, -zScale/f)

    glColor3f(0., 1., 0.)
    glLineWidth(3)
    glBegin(GL_LINE_LOOP)
    glVertex3f(0, 0, 0)
    glVertex3f(objWidth, 0, 0)
    glVertex3f(objWidth, objHeight, 0)
    glVertex3f(0, objHeight, 0)
    glEnd()
    glBegin(GL_LINE_LOOP)
    glVertex3f(0, 0, z1)
    glVertex3f(objWidth, 0, z1)
    glVertex3f(objWidth, objHeight, z1)
    glVertex3f(0, objHeight, z1)
    glEnd()
    glBegin(GL_LINE_LOOP)
    glVertex3f(0, 0, z2)
    glVertex3f(objWidth, 0, z2)
    glVertex3f(objWidth, objHeight, z2)
    glVertex3f(0, objHeight, z2)
    glEnd()

    glColor3f(1., 0., 1.) 
    glBegin(GL_LINES)
    glVertex3f(0, 0, z1)
    glVertex3f(0, 0, z2)
    glVertex3f(objWidth, 0, z1)
    glVertex3f(objWidth, 0, z2)
    glVertex3f(objWidth, objHeight, z1)
    glVertex3f(objWidth, objHeight, z2)
    glVertex3f(0, objHeight, z1)
    glVertex3f(0, objHeight, z2)
    glEnd()
    glPopMatrix()


def myPlane(minThreshold, maxThreshold):
    global objWidth, objHeight
    global minDepth, maxDepth, meanDepth

    N = 8
    f = 1000
    glLineWidth(1)
    glPushMatrix()
    glTranslatef(-objWidth/f/2, objHeight/f/2, 0)
    glScalef(1/f, -1/f, -1/f/5)
    glBegin(GL_LINES)
    glColor3f(0.98, .04, .70)
    for j in range(4):
        for i in range(N+1):
            glVertex3f(i*objWidth/N, 0, j*f)
            glVertex3f(i*objWidth/N, objHeight, j*f)
            glVertex3f(0, i*objHeight/N, j*f)
            glVertex3f(objWidth, i*objHeight/N, j*f)
    glColor3f(0.04, 0.98, .70)
    for i in range(N+1):
        glVertex3f(i*objWidth/N, 0, minThreshold)
        glVertex3f(i*objWidth/N, objHeight, minThreshold)
        glVertex3f(0, i*objHeight/N, minThreshold)
        glVertex3f(objWidth, i*objHeight/N, minThreshold)
    glColor3f(0.04, 0.70, 0.98)
    for i in range(N+1):
        glVertex3f(i*objWidth/N, 0, maxThreshold)
        glVertex3f(i*objWidth/N, objHeight, maxThreshold)
        glVertex3f(0, i*objHeight/N, maxThreshold)
        glVertex3f(objWidth, i*objHeight/N, maxThreshold)
    glEnd();    
    glPopMatrix()
    
    
def myDepth(dwidth, dheight):
    global skip, pointSize, zScale, faceType 
    global colorImage, depthColormap, convertDepthImage
    global objSx, objSy, objWidth, objHeight
    global minDepth, maxDepth

    f = 1000
    glPointSize(pointSize)
    glPushMatrix()
    glTranslatef(-objWidth/f/2, objHeight/f/2, 0)
    glScalef(1/f, -1/f, -zScale/f)

    glTranslatef(-objSx, -objSy, -meanDepth)
    if faceType == 0:
        glBegin(GL_POINTS)
        for y in range(objSy, objSy+objHeight, skip):
            for x in range(objSx, objSx+objWidth, skip):
                g = depthImage[y][x]
                if g > 9:
                    c = depthColormap[y][x]
                    glColor3ub(c[2], c[1], c[0])
                    glVertex3f(x, y, g)
        glEnd()
    elif faceType == 1:
        glBegin(GL_POINTS)
        for y in range(objSy, objSy+objHeight, skip):
            for x in range(objSx, objSx+objWidth, skip):
                g = depthImage[y][x]
                if g > 9:
                    c = colorImage[y][x]
                    glColor3ub(c[2], c[1], c[0])
                    glVertex3f(x, y, g)
        glEnd()
    elif faceType == 2:
        if(maxDepth > minDepth):
            d = maxDepth-minDepth
        else:
            d = 255    
        r = 255/d
        glBegin(GL_POINTS)
        for y in range(objSy, objSy+objHeight, skip):
            for x in range(objSx, objSx+objWidth, skip):
                g = depthImage[y][x]
                if g > 9:
                    c = int((g-minDepth)*r)
                    glColor3ub(c, c, c)
                    glVertex3f(x, y, g)
        glEnd()
    elif faceType == 3:
        if(maxDepth > minDepth):
            d = maxDepth-minDepth
        else:
            d = 255    
        r = 255/d
        for y in range(objSy, objSy+objHeight-skip, skip):
            for x in range(objSx, objSx+objWidth-skip, skip):
                if depthImage[y][x] > 9 and depthImage[y][x+skip] > 9 and \
                    depthImage[y+skip][x+skip] > 9 and depthImage[y+skip][x] > 9:
                    glBegin(GL_TRIANGLE_STRIP)
                    g = depthImage[y][x]
                    c = int((g-minDepth)*r)
                    glColor3ub(c, c, c)
                    glVertex3f(x, y, g)
                    g = depthImage[y][x+skip]
                    c = int((g-minDepth)*r)
                    glColor3ub(c, c, c)
                    glVertex3f(x+skip, y, g)
                    g = depthImage[y+skip][x]
                    c = int((g-minDepth)*r)
                    glColor3ub(c, c, c)
                    glVertex3f(x, y+skip, g)
                    g = depthImage[y+skip][x+skip]
                    c = int((g-minDepth)*r)
                    glColor3ub(c, c, c)
                    glVertex3f(x+skip, y+skip, g)
                    glEnd()        
    elif faceType == 4:
        for y in range(objSy, objSy+objHeight-skip, skip):
            for x in range(objSx, objSx+objWidth-skip, skip):
                if depthImage[y][x] > 9 and depthImage[y][x+skip] > 9 and \
                    depthImage[y+skip][x+skip] > 9 and depthImage[y+skip][x] > 9:
                    glBegin(GL_TRIANGLE_STRIP)
                    g = depthImage[y][x]
                    c = colorImage[y][x]
                    glColor3ub(c[2], c[1], c[0])
                    glVertex3f(x, y, g)
                    g = depthImage[y][x+skip]
                    c = colorImage[y][x+skip]
                    glColor3ub(c[2], c[1], c[0])
                    glVertex3f(x+skip, y, g)
                    g = depthImage[y+skip][x]
                    c = colorImage[y+skip][x]
                    glColor3ub(c[2], c[1], c[0])
                    glVertex3f(x, y+skip, g)
                    g = depthImage[y+skip][x+skip]
                    c = colorImage[y+skip][x+skip]
                    glColor3ub(c[2], c[1], c[0])
                    glVertex3f(x+skip, y+skip, g)
                    glEnd()        
    glPopMatrix()
 

def myFace(face):
    if face:
        glBegin(GL_QUADS)
        for surface in surfaces:
          x = 0
          for vertex in surface:
            x += 1    
            glColor3fv(colors[x])
            glVertex3fv(verts[vertex])
        glEnd()
    glBegin(GL_LINES)
    for edge in edges:
        for vertex in edge:
            glVertex3fv(verts[vertex])
    glEnd()

def myDisplay(minThreshold, maxThreshold):    
    global dwidth, dheight
    global eye, center, delay     
    global rotX, rotY, scale
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT) 
    glLoadIdentity()   
    if projection:
        gluLookAt(eye[0], eye[1], eye[2], center[0], center[1], center[2], 0, 1, 0)

    myMessage()
    glRotatef(rotX, 1, 0, 0)
    glRotatef(rotY, 0, 1, 0)
    glScalef(scale, scale, scale)
    # myFace()
    myDepth(dwidth, dheight)
    if displayPlane:    
        myPlane(minThreshold, maxThreshold)
    if displayAxis:
        myAxis()

    pygame.display.flip()
    pygame.time.wait(delay)


def myQuit():
    pygame.quit()
    cv2.destroyAllWindows()
    

In [16]:
def myMessage1():
    global width, height, fontSize, ym, clock   
    w, h = 7, 2.2
    glPushMatrix()
    glTranslatef(-8.2, ym-h, 0)
    glColor3f(.3, .3, .3)
    glBegin(GL_QUADS)
    glVertex3f(0, 0, 0)
    glVertex3f(w, 0, 0)
    glVertex3f(w, h, 0)
    glVertex3f(0, h, 0)
    glEnd()
    glPopMatrix()
    xmargin = 40
    ymargin = 40
    gray = 76
    drawText(xmargin, height-ymargin-fontSize, "inform(i)"+str(displayInformation)+
        " scale(z/x)"+'{:.2f}'.format(scale)+" skip(a/s)"+str(skip), gray)
    drawText(xmargin, height-ymargin-2*fontSize, "realSensor(r)"+str(displayRealSensor)+
        " heatCam(e)"+str(displayHeatCamera), gray)
    drawText(xmargin, height-ymargin-3*fontSize, "pointSize(q/w)"+str(pointSize)+
        " faceType(f)"+str(faceType), gray)
    drawText(xmargin, height-ymargin-4*fontSize, "axis(c)"+str(displayAxis)+
        " plane(v)"+str(displayPlane), gray)
    drawText(xmargin, height-ymargin-5*fontSize, "tracking(t)"+str(displayTracking)+ 
        " flir(l)"+str(debugCameraFlag)+
        " fps:" + str(int(clock.get_fps()))+"-"+str(frameCount), gray)
    drawText(xmargin, height-ymargin-6*fontSize, msg, gray)
    drawText(xmargin, height-ymargin-7*fontSize, "("+str(objSx)+","+str(objSy)+")-("+
        str(objWidth)+","+str(objHeight)+")-("+
        str(int(minDepth))+","+str(int(meanDepth))+","+str(int(maxDepth))+")", gray)
    
    
def myMessage2():
    global dwidth, dheight, imageScale, ym    
    w, h = dwidth/imageScale, dheight/imageScale
    glPushMatrix()
    glTranslatef(-8.2, -ym, 0)
    glColor3f(.3, .3, .3)
    glEnable(GL_TEXTURE_2D)
    glBindTexture(GL_TEXTURE_2D, 1)
    glBegin(GL_QUADS)
    glTexCoord2f(0.0, 1.0) 
    glVertex3f(0, 0, 0)
    glTexCoord2f(1.0, 1.0) 
    glVertex3f(w, 0, 0)
    glTexCoord2f(1.0, 0.0) 
    glVertex3f(w, h, 0)
    glTexCoord2f(0.0, 0.0) 
    glVertex3f(0, h, 0)
    glEnd()
    glPopMatrix()
    glDisable(GL_TEXTURE_2D)

    
def myMessage3():
    global dwidth, dheight, imageScale, ym    
    w, h = dwidth/imageScale, dheight/imageScale
    glPushMatrix()
    glTranslatef(8.2-w, -ym, 0)
    glColor3f(.3, .3, .3)
    glEnable(GL_TEXTURE_2D)
    glBindTexture(GL_TEXTURE_2D, 2)
    glBegin(GL_QUADS)
    glTexCoord2f(0.0, 1.0) 
    glVertex3f(0, 0, 0)
    glTexCoord2f(1.0, 1.0) 
    glVertex3f(w, 0, 0)
    glTexCoord2f(1.0, 0.0) 
    glVertex3f(w, h, 0)
    glTexCoord2f(0.0, 0.0) 
    glVertex3f(0, h, 0)
    glEnd()
    glPopMatrix()
    glDisable(GL_TEXTURE_2D)

    
def myMessage4():
    global imageScale, ym
    global resized

    s = 1
    w, h = 640/imageScale*s, 480/imageScale*s 
    updateImage(resized, dwidth, dheight, 3)
    glPushMatrix()
    glTranslatef(8.2-w, ym-h, 0)
    glColor3f(.3, .3, .3)
    glEnable(GL_TEXTURE_2D)
    glBindTexture(GL_TEXTURE_2D, 3)
    glBegin(GL_QUADS)
    glTexCoord2f(0.0, 1.0) 
    glVertex3f(0, 0, 0)
    glTexCoord2f(1.0, 1.0) 
    glVertex3f(w, 0, 0)
    glTexCoord2f(1.0, 0.0) 
    glVertex3f(w, h, 0)
    glTexCoord2f(0.0, 0.0) 
    glVertex3f(0, h, 0)
    glEnd()
    glPopMatrix()
    glDisable(GL_TEXTURE_2D)
    
    
def myMessage():
    if displayInformation:
        myMessage1()
    if displayRealSensor:
        myMessage2()
        myMessage3()
    if displayHeatCamera:
        myMessage4()

In [17]:
def myCamera():
    global capture, captureFlag 
    global frame, gray, resized
    if captureFlag == False :
        return
    if capture : 
        ret, frame = capture.read()
        if ret:
            frame = cv2.flip(frame, 1)
            gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            resized = cv2.resize(frame, (dwidth, dheight), interpolation = cv2.INTER_NEAREST) #INTER_AREA

            
def myRealSensor():
    global pipeline, align
    global dwidth, dheight, depthImage, colorImage
    global convertDepthImage, depthColormap, frameCount

    if captureFlag == False :
        return
    frames = pipeline.wait_for_frames()
    frames = align.process(frames)

    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()

    # Convert images to numpy arrays
    depthImage = np.asanyarray(depth_frame.get_data())
    colorImage = np.asanyarray(color_frame.get_data())

    # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
    convertDepthImage = cv2.convertScaleAbs(depthImage, alpha=0.03)
    depthColormap = cv2.applyColorMap(convertDepthImage, cv2.COLORMAP_JET)
    updateImage(depthColormap, dwidth, dheight, 2)
    updateImage(colorImage, dwidth, dheight, 1)



def setCamera(camId):
    global capture, fps, msg
    if camId >= 0:    
        capture = cv2.VideoCapture(camId) # camera id    
        if capture:
            capture.set(cv2.CAP_PROP_FPS, fps)
            capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280) #1920x1080 1280x720 960x540 640x360
            capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720) #640x480
            msg = "Connect Camera " + str(camId) + " ... "
            heat_width = capture.get(cv2.CAP_PROP_FRAME_WIDTH) # Frame Width
            heat_height = capture.get(cv2.CAP_PROP_FRAME_HEIGHT) # Frame Height
            heat_fps = capture.get(cv2.CAP_PROP_FPS) 
            print('connect camera ' + str(camId) + ' width: ' + str(heat_width) + ', height: ' + str(heat_height) + ', fps: ' + str(heat_fps))
        else:    
            msg = "No Camera ... "
            print('No Camera ... ')

 
def setRealSensor(): 
    # Configure depth and color streams
    global pipeline, align, dwidth, dheight, fps, depthIntrinsic
    pipeline = rs.pipeline()
    config = rs.config()

    # Get device product line for setting a supporting resolution
    pipeline_wrapper = rs.pipeline_wrapper(pipeline)
    pipeline_profile = config.resolve(pipeline_wrapper)
    device = pipeline_profile.get_device()
    device_product_line = str(device.get_info(rs.camera_info.product_line))

    found_rgb = False
    for s in device.sensors:
        if s.get_info(rs.camera_info.name) == 'RGB Camera':
            found_rgb = True
            break
    if not found_rgb:
        print("The demo requires Depth camera with Color sensor")
        exit(0)

    config.enable_stream(rs.stream.depth, dwidth, dheight, rs.format.z16, fps)

    if device_product_line == 'L500':
        config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, fps)
    else:
        config.enable_stream(rs.stream.color, dwidth, dheight, rs.format.bgr8, fps)

    align = rs.align(rs.stream.depth)
    pipeline.start(config)

    # Get the intrinsics information for calculation of 3D point
    unaligned_frames = pipeline.wait_for_frames()
    frames = align.process(unaligned_frames)
    depth = frames.get_depth_frame()
    depthIntrinsic = depth.profile.as_video_stream_profile().intrinsics

    
def stopCamera():
    global capture
    if capture:    
        capture.release()
        capture = 0

    
def stopRealSensor():
    global pipeline
    pipeline.stop()


In [18]:
def myEvent(debug, iteration, gap, minPerimeter, minThreshold, maxThreshold, margin, thresholdFlir):
    global loop, faceType, rotX, rotY, scale, fRate, msg, skip
    global mouseX, mouseY, pmouseX, pmouseY, button, dwidth, dheight
    global pointSize, displayHeatCamera, displayRealSensor, displayInformation
    global displayAxis, displayPlane, displayTracking, captureFlag, frameCount
    global objSx, objSy, objWidth, objHeight, meanDepth, debugCameraFlag
    frameCount+=1

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            loop = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                loop = False
                msg = "ESC stop..."
            elif event.key == pygame.K_SPACE:
                if captureFlag:
                    myProcess(debug, iteration, gap, minPerimeter, minThreshold, maxThreshold, margin)
                    msg = "obj capture  : (" + str(objSx) + ","  + str(objSy) + ")-(" + str(objWidth) + ","  + str(objHeight) + ")" 
                else:
                    objSx, objSy, meanDepth = 0, 0, 0
                    objWidth, objHeight = dwidth, dheight    
                    msg = "recapture  : (" + str(objWidth) + ","  + str(objHeight) + ")" 
                captureFlag = not captureFlag
                msg = "space key : " + str(captureFlag)
            elif event.key == pygame.K_c:
                msg = "start calibration ..." 
            elif event.key == pygame.K_d:
                msg = "capture flir and color images ..."
            elif event.key == pygame.K_i:
                displayInformation = not displayInformation
                msg = "i key information : " + str(displayInformation)
            elif event.key == pygame.K_e:
                displayHeatCamera = not displayHeatCamera
                msg = "e key heatCamera : " + str(displayHeatCamera)
            elif event.key == pygame.K_r:
                displayRealSensor = not displayRealSensor
                msg = "r key realSensor : " + str(displayRealSensor)
            elif event.key == pygame.K_t:
                displayTracking = not displayTracking
                msg = "t key tracking : " + str(displayTracking)
            elif event.key == pygame.K_g:
                displayAxis = not displayAxis
                msg = "g key displayAxis : " + str(displayAxis)
            elif event.key == pygame.K_v:
                displayPlane = not displayPlane
                msg = "v key displayPlane : " + str(displayPlane)
            elif event.key == pygame.K_l:
                debugCameraFlag = not debugCameraFlag
                msg = "l key debugCameraFlag : " + str(debugCameraFlag)
            elif event.key == pygame.K_f:
                faceType+=1
                if faceType > 4:
                    faceType = 0
                msg = "f key faceType : " + str(faceType)
            elif event.key == pygame.K_u:
                rotX = rotY = 0
                msg = "u key reset rot"
            elif event.key == pygame.K_z:
                scale *= 1.075
                msg = "z key scale : " + '{:.2f}'.format(scale)
            elif event.key == pygame.K_x:
                scale /= 1.075
                msg = "x key scale : " + '{:.2f}'.format(scale)
            elif event.key == pygame.K_a:
                skip+=1
                msg = "a key skip : " + str(skip)
            elif event.key == pygame.K_s:
                if skip>1:
                    skip-=1
                msg = "s key skip : " + str(skip)
            elif event.key == pygame.K_q:
                pointSize+=1
                msg = "q key pointSize : " + str(pointSize)
            elif event.key == pygame.K_w:
                if pointSize>1:
                    pointSize-=1
                msg = "w key pointSize : " + str(pointSize)
        elif event.type == pygame.MOUSEBUTTONDOWN:
            mouseX, mouseY = pygame.mouse.get_pos()
            pmouseX, pmouseY, button = mouseX, mouseY, True
            msg = "mouse down (" + str(mouseX) + "," + str(mouseY)+ ") " + str(button)
        elif event.type == pygame.MOUSEMOTION:
            mouseX, mouseY = pygame.mouse.get_pos()
            if button == True:
                rotX -= (pmouseY - mouseY)*fRate
                rotY -= (pmouseX - mouseX)*fRate
                pmouseX, pmouseY = mouseX, mouseY
        elif event.type == pygame.MOUSEBUTTONUP:
            mouseX, mouseY = pygame.mouse.get_pos()
            button = False
            msg = "mouse up (" + str(mouseX) + "," + str(mouseY)+ ") " + str(button)   
       

In [19]:
def main(camId, dwidth, dheight, debug, iteration, gap, minPerimeter, 
    minThreshold, maxThreshold, margin, thresholdFlir):
    myInit(dwidth, dheight)
    myReshape()
    setCamera(camId)
    setRealSensor()
    while(loop):
        myCamera()
        myRealSensor()
        myEvent(debug, iteration, gap, minPerimeter, 
            minThreshold, maxThreshold, margin, thresholdFlir)    
        myDisplay(minThreshold, maxThreshold)
        clock.tick(fps)
  
    stopRealSensor()
    stopCamera()
    myQuit()

In [20]:
debug, camId = 0, 0                     # 1:display log 2:cv2.imshow 3:cv2.imwrite 
iteration, gap = 3, 1                   # no of morpological operation 
minThreshold, maxThreshold = 100, 1000  # min, max depth threshold
margin, minPerimeter = 3, 1000          # filling depth data - median filter margin, minimum blob perimeter
thresholdFlir = 200                     # Flir Image Green frame threshold    
main(camId, 1280, 720, debug, iteration, gap, minPerimeter, 
    minThreshold, maxThreshold, margin, thresholdFlir)


connect camera 0 width: 1280.0, height: 720.0, fps: 30.0
